# Infraestructure Manager revenue maximization with GSA

## 0. Load libraries

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from benchmarks.railway import Solution, RevenueMaximization
from src.entities import GSA

from typing import List, Mapping, Union

## 1. Define corridor

As an example, we will use the Spanish south high-speed railway corridor.



In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                               },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

### 1.1. Get lines from corridor

In [3]:
# Get lines from corridor

def get_lines(corridor: Mapping[str, Mapping],
              path: Union[List, None]=None
              ) -> List[List[str]]:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

### 1.2. Sample a random line, and a random route from the line

In [4]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA']
Sampled route: ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA']


### 1.3 Generate random timetable for a route

- Times in minutes.
- Initial time is randomized between 0 and 24*60 (24 hours).
- The time between stations is also randomized between 30 and 120 minutes.
- The time to dwell at each station is randomized between 2 and 8 minutes.

In [5]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'MAD': (1174, 1174),
 'CIU': (1259, 1263),
 'COR': (1311, 1315),
 'PGE': (1366, 1373),
 'ANT': (1417, 1419),
 'GRA': (1501, 1501)}

## 2. Generate as many requested services as needed

In [6]:
# Generate random requested timetable in corridor for a day t

def get_schedule_request(n_services: int) -> dict:
    """
    Generate random timetable
    
    Args:
        n_services (int): number of services
    
    Returns:
        dict: timetable
    """
    return {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}

# Generate random schedule
schedule = get_schedule_request(3)
schedule

{1: {'COR': (413, 413),
  'PGE': (476, 478),
  'ANT': (579, 585),
  'MAL': (612, 612)},
 2: {'ANT': (347, 347), 'MAL': (423, 423)},
 3: {'MAD': (1423, 1423),
  'CIU': (1471, 1475),
  'COR': (1562, 1564),
  'PGE': (1646, 1650),
  'ANT': (1685, 1687),
  'MAL': (1774, 1774)}}

## 3. Define feasibility function

In [31]:
schedule = get_schedule_request(3)
schedule

{1: {'COR': (966, 966), 'SEV': (1020, 1027), 'CAD': (1074, 1074)},
 2: {'MAD': (995, 995),
  'CIU': (1075, 1080),
  'COR': (1120, 1123),
  'PGE': (1201, 1207),
  'ANT': (1252, 1254),
  'GRA': (1318, 1318)},
 3: {'COR': (73, 73), 'PGE': (147, 150), 'ANT': (226, 228), 'MAL': (322, 322)},
 4: {'PGE': (1166, 1166), 'ANT': (1198, 1202), 'MAL': (1280, 1280)},
 5: {'PGE': (1018, 1018), 'ANT': (1068, 1074), 'MAL': (1182, 1182)},
 6: {'CIU': (1343, 1343),
  'COR': (1442, 1447),
  'PGE': (1485, 1492),
  'ANT': (1578, 1582),
  'MAL': (1697, 1697)},
 7: {'COR': (1423, 1423),
  'PGE': (1504, 1508),
  'ANT': (1622, 1629),
  'GRA': (1735, 1735)},
 8: {'COR': (290, 290), 'SEV': (350, 355), 'CAD': (411, 411)},
 9: {'CIU': (689, 689),
  'COR': (788, 795),
  'SEV': (891, 897),
  'CAD': (979, 979)},
 10: {'PGE': (109, 109), 'ANT': (217, 221), 'MAL': (273, 273)},
 11: {'PGE': (988, 988), 'ANT': (1027, 1029), 'GRA': (1072, 1072)},
 12: {'PGE': (1168, 1168), 'ANT': (1265, 1272), 'MAL': (1350, 1350)},
 13: {'

In [7]:
# Dummy schedule
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [8]:
from scipy.stats import loguniform

np.random.seed(seed=22)

def get_revenue_behaviour(schedule: dict) -> dict:
    """
    Get revenue behaviour
    
    Args:
        schedule (dict): schedule
    
    Returns:
        dict: revenue behaviour
    """
    revenue = {}
    bias = [0.2, 0.35, 0.1]
    for service in schedule:
        b = np.random.choice(bias)
        base_price = 55 * len(schedule[service])
        canon = base_price + b * base_price
        k = loguniform.rvs(0.01, 100, 1)
        max_penalty = canon * 0.4
        dt_penalty = max_penalty * 0.35
        tt_penalty = (max_penalty - dt_penalty) / (len(schedule[service]) - 1)
        revenue[service] = {'canon': canon, 'k': k, 'dt_max_penalty': dt_penalty, 'tt_max_penalty': tt_penalty}
    return revenue

revenue = get_revenue_behaviour(schedule)
revenue

{1: {'canon': 222.75,
  'k': 7.424686268142557,
  'dt_max_penalty': 31.185000000000002,
  'tt_max_penalty': 28.957500000000003},
 2: {'canon': 264.0,
  'k': 1.4810078247291318,
  'dt_max_penalty': 36.96,
  'tt_max_penalty': 22.880000000000006},
 3: {'canon': 181.5,
  'k': 14.186717826602765,
  'dt_max_penalty': 25.41,
  'tt_max_penalty': 23.595000000000006}}

In [9]:
sm = RevenueMaximization(schedule, revenue, safe_headway=10)

In [10]:
gsa_algo = GSA(objective_function=sm.get_fitness_gsa,
               is_feasible=sm.feasible_services_times,
               r_dim=len(sm.real_boundaries),
               d_dim=0,
               boundaries=sm.boundaries)

In [11]:
import time

gsa_algo.set_seed(seed=28)

pop_size = 5

start = time.time()
training_history = gsa_algo.optimize(population_size=pop_size,
                                     iters=50,
                                     chaotic_constant=True,
                                     repair_solution=True,
                                     initial_population=sm.get_initial_population(pop_size))

print(f"Elapsed time: {time.time() - start}")

Initial population: [<src.entities.Solution object at 0x15fde6fa0>, <src.entities.Solution object at 0x15fddc640>, <src.entities.Solution object at 0x15fddc850>, <src.entities.Solution object at 0x106377bb0>, <src.entities.Solution object at 0x15fe0aac0>]
GSA is optimizing  "get_fitness_gsa"
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 1 the best fitness is 517.0149775314684']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 2 the best fitness is 530.855182872092']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 3 the best fitness is 542.6469392432284']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 4 the best fitness is 565.6612579772318']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution

In [12]:
training_history

,Iteration,Fitness,Accuracy,ExecutionTime,Discrete,Real
0,0,517.014978,0,0.003048,[],"[-4.98, 157.83, 161.92, 189.92, 11.98, 35.76, ..."
1,1,530.855183,0,0.085591,[],"[-2.7546657683245406, 157.40093372659027, 161...."
2,2,542.646939,0,0.165635,[],"[-1.4908902847345742, 157.03956686173672, 161...."
3,3,565.661258,0,0.218120,[],"[-0.08372446694929914, 152.02590705979338, 157..."
4,4,565.661258,0,0.242157,[],"[5.670225709399254, 148.90734406439034, 159.04..."
5,5,565.661258,0,0.312635,[],"[5.670225709399254, 148.90734406439034, 159.04..."
6,6,565.661258,0,0.359963,[],"[5.670225709399254, 148.90734406439034, 159.04..."
7,7,566.497557,0,0.363831,[],"[-9.771727430351225, 145.23158773598314, 153.9..."
8,8,568.721232,0,0.367587,[],"[-7.794995179434571, 143.10007102447446, 154.2..."
9,9,568.721232,0,0.370925,[],"[-9.92970600467806, 143.22751590497882, 153.84..."


In [13]:
# Get last value in column 'Real' of training_history
best_solution = training_history.iloc[-1]['Real']

In [16]:
sm.update_feasible_schedules(Solution(real=np.array(best_solution, dtype=float), discrete=np.array([])))
sm.feasible_schedules

[[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 1]]

In [17]:
S_i = sm.get_best_schedule(Solution(real=best_solution, discrete=np.array([])))
print(S_i)

sm.get_revenue(Solution(real=best_solution, discrete=S_i))

[1 1 1]


591.2527108040338

In [18]:
sm.get_best_schedule(Solution(real=best_solution, discrete=np.array([])))

array([1, 1, 1])

In [19]:
sum([revenue[service]['canon'] for service in revenue])

668.25

In [20]:
training_history

,Iteration,Fitness,Accuracy,ExecutionTime,Discrete,Real
0,0,517.014978,0,0.003048,[],"[-4.98, 157.83, 161.92, 189.92, 11.98, 35.76, ..."
1,1,530.855183,0,0.085591,[],"[-2.7546657683245406, 157.40093372659027, 161...."
2,2,542.646939,0,0.165635,[],"[-1.4908902847345742, 157.03956686173672, 161...."
3,3,565.661258,0,0.218120,[],"[-0.08372446694929914, 152.02590705979338, 157..."
4,4,565.661258,0,0.242157,[],"[5.670225709399254, 148.90734406439034, 159.04..."
5,5,565.661258,0,0.312635,[],"[5.670225709399254, 148.90734406439034, 159.04..."
6,6,565.661258,0,0.359963,[],"[5.670225709399254, 148.90734406439034, 159.04..."
7,7,566.497557,0,0.363831,[],"[-9.771727430351225, 145.23158773598314, 153.9..."
8,8,568.721232,0,0.367587,[],"[-7.794995179434571, 143.10007102447446, 154.2..."
9,9,568.721232,0,0.370925,[],"[-9.92970600467806, 143.22751590497882, 153.84..."
